# Accompanying notebook to Optimization Stories in Logistics and Transportation

By Joaquim Gromicho, ORTEC & The Analytics Academy, 2020

As we know, Caroline produces football and golf trophies using two types of ornamental balls, plaques and wood. 

Each trophy has the following consumption of materials:

| Trophy | Wood in meter | Football | Golf ball | Plaque |
|:-------|--------------:|---------:|----------:|-------:|
|Football|           0.4 |        1 |           |      1 |
|Golf    |           0.2 |          |         1 |      1 |

Caroline conducted a data analysis which lead to the following prediction of monthly demands for her trophies: 

| Trophy | Jan | Feb | Mar | Apr | May | Jun | Jul | Aug | Sep | Oct | Nov | Dec |
|:-------|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
|Football|  88 | 125 | 260 | 217 | 238 | 286 | 248 | 238 | 265 | 293 | 259 | 244 |
|Golf    |  47 |  62 |  81 |  65 |  95 | 118 |  86 |  89 |  82 |  82 |  84 | 66  |

Caroline hired Hilda to plan the materials. 
Hilda may buy from each of three suppliers and may also stock materials. 

The suppliers can deliver the following materials:
 - A: **footballs**, **golf balls** and **plaques**
 - B: **wood**
 - C: all of the above

Wood should be acquired in multiples of 10 meter, since it is delivered in poles of 10 meter. 
Ornaments such as balls and plaques may be acquired in any number, but the price is in batches of 100. Meaning that 30 footballs with 10 golf balls and 50 plaques costs as much as 1 football but half of 30 footballs with 30 golf balls and 50 plaques. 
Furthermore, supplier C sells all materials and offers a discount if purchased together: 10 meter of wood and a batch of ornaments cost just 7. This set price is only applied to pairs, meaning that 10 meter of wood and 2 batches cost 13.

The prices are as follows in &euro;:

|Supplier|Wood per 10 meter pole|Batch of ornaments|Together|
|:-------|---------------------:|-----------------:|-------:|
| A      |                    - |                5 |      - |
| B      |                    3 |                - |      - |
| C      |                    4 |                6 |      7 |


When stocking materials, the inventory costs are as follows per month:

|Wood per meter|Football per unit|Golf per unit|Plaque per unit|
|---:|-------:|---:|-----:|
| 0.1|   0.02 |0.02| 0.02 |

The holding price of wood is per meter and the wood stocked is rounded up to full meters, meaning that 12 decimeters pay for 2 meters. 

The capacity limitations of the warehouse allow for a maximum of 1000 meter of wood in stock at any moment.
There are no practical limitations to the amount of ornaments in stock.

As you recall, Caroline has the following stock at the moment:

|Wood|Football|Golf|Plaque|
|---:|-------:|---:|-----:|
| 480|   1000 |1500| 1750 |

Caroline would like to have at least the following stock at the end of the year:

|Wood|Football|Golf|Plaque|
|---:|-------:|---:|-----:|
| 200|    500 | 500| 1000 |

Please help Hilda to model the material planning for Caroline and solve it with the data above. 

Note that Hilda aims at minimizing the acquisition and holding costs of the materials while meeting the required quantities for production. 
The production is made to order, meaning that no inventory of trophies is kept.

In [1]:
import pandas
demand = pandas.read_csv( "demand.csv", index_col='Trophy' )
demand

Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
Trophy                                                              
Football   88  125  260  217  238  286  248  238  265  293  259  244
Golf       47   62   81   65   95  118   86   89   82   82   84   66

## A simple dataframe with the consumptions

In [2]:
use = dict()
use['Football'] = { 'football' : 1, 'plaque' : 1, 'wood' : 4 }
use['Golf'] = { 'golf ball' : 1, 'plaque' : 1, 'wood' : 2 }
use = pandas.DataFrame.from_dict( use ).fillna(0).astype( int )
use 

Football  Golf
football          1     0
plaque            1     1
wood              4     2
golf ball         0     1

## From products to materials: a simple matrix multiplication

In [3]:
demand = use.dot( demand )
demand

Jan  Feb   Mar  Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec
football    88  125   260  217   238   286   248   238   265   293   259   244
plaque     135  187   341  282   333   404   334   327   347   375   343   310
wood       446  624  1202  998  1142  1380  1164  1130  1224  1336  1204  1108
golf ball   47   62    81   65    95   118    86    89    82    82    84    66

In [4]:
from pyomo.environ import ConcreteModel, Var, Objective, minimize, \
                          Constraint, ConstraintList,              \
                          NonNegativeReals, NonNegativeIntegers

model = ConcreteModel("Hilda's material scheduling")  

model.periods = demand.keys()
model.materials = demand.index

In [5]:
model.existing = {'football' : 1000, 'golf ball': 1500, 'plaque': 1750, 'wood' : 4800 }
model.desired = {'football' : 500, 'golf ball': 500, 'plaque': 1000, 'wood' : 2000 }

In [6]:
model.suppliers        = ['A','B','C']
model.supplyingPoles   = [ 'B', 'C' ]
model.supplyingBatches = [ 'A', 'C' ]
model.givingDiscount   = [ 'C' ]

# we express prices in cents to preserve integers
model.priceBatch = { 'A': 500, 'C':600 }
model.pricePole  = { 'B': 300, 'C':400 }
model.discount   = model.priceBatch['C']+model.pricePole['C']-700

model.batchSize = 100

model.unitaryHoldingCosts = { 'wood': 10, 'football' : 2, 'golf ball': 2, 'plaque': 2 }
model.unitaryMaterials = [ 'football', 'golf ball', 'plaque']

In [7]:
model.stock     = Var(model.materials,model.periods, within=NonNegativeReals )
model.buy       = Var(model.supplyingBatches,model.unitaryMaterials,model.periods, within=NonNegativeReals )
model.batches   = Var(model.supplyingBatches,model.periods, within=NonNegativeIntegers )
model.poles     = Var(model.supplyingPoles,model.periods, within=NonNegativeIntegers )
model.pairs     = Var(model.periods, within=NonNegativeIntegers )
model.meters    = Var(model.periods, within=NonNegativeIntegers )

model.obj = Objective( expr = sum( model.priceBatch[b] * model.batches[b,i] for b in model.supplyingBatches for i in model.periods )        \
                            + sum( model.pricePole[p] * model.poles[p,i] for p in model.supplyingPoles for i in model.periods )             \
                            - sum( model.discount * model.pairs[i] for i in model.periods )                                                 \
                            + sum( model.unitaryHoldingCosts['wood'] * model.meters[i] for i in model.periods )                             \
                            + sum( model.unitaryHoldingCosts[m] * model.stock[m,i] for m in model.unitaryMaterials for i in model.periods ) \
                     , sense = minimize
                     )

def PairsInBatches( model, i ):
    return model.pairs[i] <= model.batches['C',i]    
def PairsInPoles( model, i ):
    return model.pairs[i] <= model.poles['C',i]    
def WoodStock( model, i ):
    return model.stock['wood',i] <= 10*model.meters[i]
def WoodStockMax( model, i ):
    return model.stock['wood',i] <= 10000
def InBatches( model, i, b ):
    return sum( model.buy[b,m,i] for m in model.unitaryMaterials ) <= model.batchSize*model.batches[b,i]

model.pairsInBatches = Constraint( model.periods, rule = PairsInBatches )
model.pairsInPoles   = Constraint( model.periods, rule = PairsInPoles )
model.woodStock      = Constraint( model.periods, rule = WoodStock )
model.woodStockMax   = Constraint( model.periods, rule = WoodStockMax )
model.inBatches      = Constraint( model.periods, model.supplyingBatches, rule = InBatches )

model.constraints = ConstraintList()  # Create a set of constraints

first = model.periods[0]
last  = model.periods[-1]

for m in model.unitaryMaterials:
    model.constraints.add( model.stock[m,first] == model.existing[m] + sum(model.buy[b,m,first] for b in model.supplyingBatches) - demand.loc[m,first] )
    for prev,current in zip(model.periods,model.periods[1:]):
        model.constraints.add( model.stock[m,current] == model.stock[m,prev] + sum(model.buy[b,m,current] for b in model.supplyingBatches) - demand.loc[m,current] )
    model.constraints.add( model.stock[m,last] >= model.desired[m] )
    
model.constraints.add( model.stock['wood',first] == model.existing['wood'] + 100*sum(model.poles[p,first] for p in model.supplyingPoles) - demand.loc['wood',first] )
for prev,current in zip(model.periods,model.periods[1:]):
    model.constraints.add( model.stock['wood',current] == model.stock['wood',prev] + 100*sum(model.poles[p,current] for p in model.supplyingPoles) - demand.loc['wood',current] )
model.constraints.add( model.stock['wood',last] >= model.desired['wood'] )

neos = False
if neos:
    from pyomo.environ import SolverManagerFactory
    %time results = SolverManagerFactory('neos').solve( model, opt='cplex')
else:
    from pyomo.opt import SolverFactory
    %time results = SolverFactory('gurobi').solve(model)

Wall time: 465 ms


In [8]:
def ShowTriple( X, I,J,K, names ):
    import pandas
    index = pandas.MultiIndex.from_product([I,J], names=names)
    return pandas.DataFrame.from_records( [ [ X[i,j,k].value for k in K ] for i in I for j in J ],index=index,columns=K )
def ShowDouble( X, I,J ):
    import pandas
    return pandas.DataFrame.from_records( [ [ X[i,j].value for j in J ] for i in I ], index=I, columns=J )
def ShowSingle( X, I ):
    import pandas
    return pandas.DataFrame( [ X[i].value for i in I ], index=I ).T

In [9]:
if  results.solver.termination_condition == 'optimal':
    import numpy
    numpy.set_printoptions(edgeitems=3,infstr='inf',linewidth=200, nanstr='nan', precision=15, suppress=False, threshold=1000, formatter=None)
    value   = model.obj.expr()
    Batches = [ [ model.batches[b,i].value for i in model.periods ] for b in model.supplyingBatches ]
    Poles   = [ [ model.poles[b,i].value for i in model.periods ] for b in model.supplyingPoles ]
    Pairs   = [ model.pairs[i].value for i in model.periods ] 
    Buy     = [ [ [ model.buy[s,m,i].value for i in model.periods ] for m in model.unitaryMaterials ] for s in model.supplyingBatches ]
    Stock   = [ [ model.stock[m,i].value for i in model.periods ] for m in model.materials ]
    Meters  = [ model.meters[i].value for i in model.periods ]
    
    def months( before='', after='', length=5 ):
        return ''.join( ['{:>{l}s}'.format(before,l=length)] + [ '{:>{l}s}'.format(m,l=length) for m in calendar.month_abbr[1:] + ['{:>{l}s}'.format(after,l=length)] ] )
        
    def describe( list, decimals=0 ):
        formattedList = [ '{:5.{prec}f}'.format( member + 0, prec=decimals ) for member in list]
        return ''.join(formattedList)
        
    print('')
    print( 'Costs : ', numpy.around( value/100, 3 ) )
    print('')
    print( '{:>13s}'.format('periods') + ''.join( [ '{:>{l}s}'.format(p,l=5) for p in model.periods ] ) )
    
    print( 'buy')
    for s,supplier in enumerate(model.supplyingBatches):
        print(supplier)
        for i,m in enumerate(model.unitaryMaterials):
            print('{:>13s}'.format(m) + describe( Buy[s][i] ) )
    print( 'batches')
    for s,supplier in enumerate(model.supplyingBatches):
        print('{:>13s}'.format(supplier) + describe( Batches[s] ) )
    print( 'poles' )
    for s,supplier in enumerate(model.supplyingPoles):
        print('{:>13s}'.format(supplier) + describe( Poles[s] ) )
        
    print( '' )
    print('{:>13s}'.format('pairs from C') + describe( Pairs ) )
    print( '' )
            
    import calendar 
    print( 'stock')
    print( '{:>12s}'.format('material') + months( before='from', after='to') )
    for j,m in enumerate(model.materials):
        print( '{:>12s}'.format(m) + describe( [model.existing[m]] + Stock[j] + [model.desired[m]] ) )
       
    print( '' )
    print('{:>17s}'.format('full wood meters') + describe( Meters ) )


Costs :  1102.16

      periods  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
buy
A
     football    0    0    0    0    0    0    0    0    0    0    0    0
    golf ball    0    0    0    0    0    0    0    0    0    0    0    0
       plaque    0    0    0    0    0    0    0    0    0    0    0    0
C
     football    0    0    0    0    0  229  233  238  265  293  313  690
    golf ball    0    0    0    0    0    0    0    0    0    0    0    0
       plaque    0    0    0    0    0    0  267  362  335  407  287 1310
batches
            A    0    0    0    0    0    0    0    0    0    0    0    0
            C    0    0    0    0    0    3    5    6    6    7    6   20
poles
            B    0    0    0    0    0    7    7    5    7    6    6   11
            C    0    0    0    0    0    3    5    6    6    7    6   20

 pairs from C    0    0    0    0    0    3    5    6    6    7    6   20

stock
    material from  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep 

In [10]:
ShowTriple( model.buy, model.supplyingBatches, model.unitaryMaterials, model.periods, ['supplier','materials'] ).astype(int)

Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  \
supplier materials                                                          
A        football     0    0    0    0    0    0    0    0    0    0    0   
         golf ball    0    0    0    0    0    0    0    0    0    0    0   
         plaque       0    0    0    0    0    0    0    0    0    0    0   
C        football     0    0    0    0    0  228  233  238  265  293  312   
         golf ball    0    0    0    0    0    0    0    0    0    0    0   
         plaque       0    0    0    0    0    0  266  362  335  406  287   

                     Dec  
supplier materials        
A        football      0  
         golf ball     0  
         plaque        0  
C        football    690  
         golf ball     0  
         plaque     1310

In [11]:
ShowDouble( model.stock, model.materials, model.periods ).astype(int)

Jan   Feb   Mar   Apr   May   Jun  Jul  Aug  Sep  Oct  Nov   Dec
football    912   787   527   310    72    14    0    0    0    0   53   500
plaque     1615  1428  1087   805   472    68    0   35   23   55    0  1000
wood       4354  3730  2528  1530   388     8   44   14   90   54   50  2042
golf ball  1453  1391  1310  1245  1150  1032  946  857  775  693  609   543

In [12]:
ShowDouble( model.batches, model.supplyingBatches, model.periods ).astype(int)

Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
A    0    0    0    0    0    0    0    0    0    0    0    0
C    0    0    0    0    0    3    5    6    6    7    6   20

In [13]:
ShowSingle( model.pairs, model.periods ).astype(int)

Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
0    0    0    0    0    0    3    5    6    6    7    6   20

In [14]:
ShowSingle( model.meters, model.periods ).astype(int)

Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec
0  436  373  253  153   39    1    5    2    9    6    5  205